In [18]:
import numpy as np


predictions = np.load("predictions.npy")
labels = np.load("labels.npy")


In [19]:
from transformers import AutoTokenizer
from datasets import load_metric
import numpy as np
import constants
import nltk


def compute_metrics(predictions, labels):
    # Evtl später übertragen
    tokenizer = AutoTokenizer.from_pretrained(constants.MODEL_NAME_TASD)
    metric = load_metric("rouge")
    print(predictions)

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(
        predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                     for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    for i in range(10):
        print("label:", decoded_labels[i], len(decoded_labels[i]))
        print("prediction:", decoded_preds[i], len(decoded_preds[i]), type(decoded_preds[i]))



    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                       for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [20]:
compute_metrics(predictions, labels)

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[[    0  4670   229 ...  -100  -100  -100]
 [    0  1387   229 ...  -100  -100  -100]
 [    0 21484    49 ...  -100  -100  -100]
 ...
 [    0  4670   229 ...  -100  -100  -100]
 [    0 11722   229 ...  -100  -100  -100]
 [    0 21484    49 ...  -100  -100  -100]]
label: Preis ist schlecht weil Geld schlecht ist [SEP] 47
prediction: Preis ist schlecht weil Geld schlecht ist [SEP] 47 <class 'str'>
label: Service ist schlecht weil Kellner schlecht ist [SEP] 52
prediction: Service ist schlecht weil Kellner schlecht ist [SEP] 52 <class 'str'>
label: Allgemeiner Eindruck ist gut weil war wunderbar gut ist [SEP] Allgemeiner Eindruck ist gut weil es gut ist [SEP] 112
prediction: Allgemeiner Eindruck ist gut weil war wunderbar gut ist [SEP] Allgemeiner Eindruck ist gut weil es gut ist [SEP] 112 <class 'str'>
label: Service ist schlecht weil es schlecht ist [SEP] 47
prediction: Service ist schlecht weil es schlecht ist [SEP] 47 <class 'str'>
label: Allgemeiner Eindruck ist gut weil es gut ist [S

{'rouge1': 100.0,
 'rouge2': 100.0,
 'rougeL': 100.0,
 'rougeLsum': 100.0,
 'gen_len': 19.39}

In [21]:
"Service ist ok weil Personal ok ist [SSEP] Ambiente ist schlecht weil U"
"Service ist schlecht weil Service schlecht ist [SSEP] Essen ist gut weil Suppe gut ist"

'Service ist schlecht weil Service schlecht ist [SSEP] Essen ist gut weil Suppe gut ist'